# Libs and Inputs

In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import time
from tqdm import tqdm
from GetIdxOutRadious import GetIdxOutRadious

LCOE_Max=range(500,50,-2) #LCOEs investigated
Radious=40 #Radious for the energy collection system

ShapeFileCoast="./GEO_data/ne_10m_coastline.shp"
ShapeFileStates="./GEO_data/ne_10m_admin_1_states_provinces_lines.shp"

WindKiteTrsData=np.load('PreprocessedData.npz',allow_pickle=True)

ResultsFileName='PortfolioOptimizationWindKite'

# Prepare Data For the Optimization Model

In [2]:
#Wind Data
WindEnergy=WindKiteTrsData['WindEnergy']
WindLatLong=WindKiteTrsData['WindLatLong']
AnnualizedCostWind=WindKiteTrsData['AnnualizedCostWind']
MaxNumWindPerSite=WindKiteTrsData['MaxNumWindPerSite']

#Kite Data
KiteEnergy=WindKiteTrsData['KiteEnergy']
KiteLatLong=WindKiteTrsData['KiteLatLong']
AnnualizedCostKite=WindKiteTrsData['AnnualizedCostKite']
MaxNumKitesPerSite=WindKiteTrsData['MaxNumKitesPerSite']

#Transmission Data
AnnualizedCostTransmission=WindKiteTrsData['AnnualizedCostTransmission']
TransLatLong=WindKiteTrsData['TransLatLong']
EfficiencyTransmission=WindKiteTrsData['EfficiencyTransmission']
MaxPowerTransmission=WindKiteTrsData['MaxPowerTransmission']

TimeStepHours=WindKiteTrsData['TimeStepHours'] #Number of hours for each time step

In [3]:
#Vectorize maximum number of turbines per site location per technology
Nu=np.concatenate((MaxNumWindPerSite,MaxNumKitesPerSite))

#Vectorize annualized cost for each site location and per technology
AnnCost=np.concatenate((AnnualizedCostWind,AnnualizedCostKite)) #Annualized cost [$/Year]

#Vectorize energy generation in each site location and energy resource
EnergyGeneration=np.concatenate((WindEnergy,KiteEnergy),axis=0) #MW ()

NumWindSites=WindEnergy.shape[0]
NumKiteSites=KiteEnergy.shape[0]

NumTrasmissionSites=TransLatLong.shape[0]

NumTimeSteps=WindEnergy.shape[1]

# Build Optimization Model Structure

In [4]:
MILP = ConcreteModel()

# Create Sets
MILP.SiteWind = RangeSet(0,NumWindSites-1)
MILP.SiteKite = RangeSet(0,NumKiteSites-1)
MILP.SiteTrs  = RangeSet(0,NumTrasmissionSites-1)

MILP.TimeSteps = RangeSet(0,NumTrasmissionSites-1)

# Create Variables
MILP.Y_Wind = Var(MILP.SiteWind, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
MILP.Y_Kite = Var(MILP.SiteKite, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location

MILP.s = Var(MILP.SiteTrs, domain=Binary)# Binary variable to track the center of the energy collection system
MILP.Delta = Var(MILP.TimeSteps, domain=NonNegativeReals) #Curtailment variable

#Objective Function
def objective_rule(MILP):   
    EGWind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite) #Energy generation from kite turbines

    TotalCurtailment=sum(MILP.Delta[i] for i in MILP.TimeSteps)/TimeStepHours #Total curtailment

    Obj=(EGWind+EGKite-TotalCurtailment)*24*365#Mwh/year
    return Obj

MILP.OBJ = Objective(rule = objective_rule, sense=maximize)

#Constraints

#Maximum number of turbines per site location wind
def MaxTurbinesCell_Wind_rule(MILP,i):
    return MILP.Y_Wind[i]<=MaxNumWindPerSite[i]
MILP.Turbines_Cell_Wind = Constraint(MILP.SiteWind, rule=MaxTurbinesCell_Wind_rule)

#Maximum number of turbines per site location kite
def MaxTurbinesCell_Kite_rule(MILP,i):
    return MILP.Y_Kite[i]<=MaxNumKitesPerSite[i]
MILP.Turbines_Cell_Kite = Constraint(MILP.SiteKite, rule=MaxTurbinesCell_Kite_rule)


#Curtailment constraint
def Curtailment_rule(MILP,t):
    EGWind=sum(MILP.Y_Wind[i]*WindEnergy[i,t] for i in MILP.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP.Y_Kite[i]*KiteEnergy[i,t] for i in MILP.SiteKite) #Energy generation from kite turbines

    return -MILP.Delta[t]+EGWind+EGKite<=MaxPowerTransmission

MILP.Curtailment = Constraint(MILP.TimeSteps, rule=Curtailment_rule)


#---Choose center collection system - Start
MILP.ChooseOneCircle= Constraint(expr=sum(MILP.s[i] for i in MILP.SiteTrs)==1)

#Get the sites that are out of the radious of the center of the collection system
IdxOutWind=GetIdxOutRadious(TransLatLong, WindLatLong,Radious)
IdxOutKite=GetIdxOutRadious(TransLatLong, KiteLatLong,Radious)

def MaximumRadious(MILP,i):  
    SumWind_s=sum(MILP.Y_Wind[j] for j in IdxOutWind[i])
    SumKite_s=sum(MILP.Y_Kite[j] for j in IdxOutKite[i])

    return SumWind_s+SumKite_s<=(1-MILP.s[i])*300 # 300 is a big M for the total number of turbines installed       

MILP.Maximum_Radious = Constraint(MILP.SiteTrs, rule=MaximumRadious)
#---Choose center collection system - End

#LCOE Target
def LCOETarget_S1(MILP,LCOE_Max):  
    EG_Wind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind)
    EG_Kite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite)
    TotalCurtailment=sum(MILP.Delta[i] for i in MILP.TimeSteps)/TimeStepHours #Total curtailment
    MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

    Cost_Wind=sum(MILP.Y_Wind[i]*AnnualizedCostWind[i] for i in MILP.SiteWind)
    Cost_Kite=sum(MILP.Y_Kite[i]*AnnualizedCostKite[i] for i in MILP.SiteKite)
    Cost_Transmission=sum(MILP.s[i]*AnnualizedCostTransmission[i] for i in MILP.SiteTrs)
    Cost=Cost_Wind+Cost_Kite+Cost_Transmission

    return Cost<=LCOE_Max*MWh # 300 is a big M for the total number of turbines installed       

#

opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.05
#opt.options['max_iter'] = 500

In [5]:
SaveFeasibility, Save_LCOETarget, Save_LCOE_Achieved, SaveTotalMWh = list(), list(), list(), list()
SaveYWind, SaveYKite, SaveYTrans, SaveCurtail = list(), list(), list(), list()

LowestLCOE=10**10
for LCOE_Idx in tqdm(range(len(LCOE_Max))):
    LCOETarget=LCOE_Max[LCOE_Idx]
    
    if LCOETarget<LowestLCOE:    
        Bypass=0
        #Upperbound For the LCOE Activate Constraint
        LCOE_Target=LCOETarget_S1(MILP,LCOETarget)
        MILP.LCOE_Target = Constraint(rule=LCOE_Target)
        print("Running Model With LCOE= %.2f" % LCOETarget)
        
        try:
            results=opt.solve(MILP, tee=True)
        except:
            Bypass=1
            MILP.del_component(MILP.LCOE_Target)  
    
        if Bypass==0:
            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                SaveFeasibility.append(1)
                Save_LCOETarget.append(LCOETarget)
                
                Optimal_Y_Kite =np.array([MILP.Y_Kite.get_values()[j] for j in MILP.SiteKite])
                Optimal_Y_Wind =np.array([MILP.Y_Wind.get_values()[j] for j in MILP.SiteWind])
                Optimal_Y_Trans=np.array([MILP.s.get_values()[j] for j in MILP.SiteTrs])
                Curtailment=np.array([MILP.Delta.get_values()[j] for j in MILP.TimeSteps])

                SaveYWind.append(Optimal_Y_Wind)
                SaveYKite.append(Optimal_Y_Kite)
                SaveYTrans.append(Optimal_Y_Trans)
                SaveCurtail.append(Curtailment)

                #Current LCOE
                EG_Wind=np.sum(Optimal_Y_Wind*np.average(WindEnergy,axis=1)) #MW avg
                EG_Kite=np.sum(Optimal_Y_Kite*np.average(KiteEnergy,axis=1)) #MW avg
                TotalCurtailment=np.sum(Curtailment)
                MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

                Cost_Wind  = sum(Optimal_Y_Wind*AnnualizedCostWind)
                Cost_Kite  = sum(Optimal_Y_Kite*AnnualizedCostKite)
                Cost_Trans = sum(Optimal_Y_Trans*AnnualizedCostTransmission)

                Cost=Cost_Wind+Cost_Kite+Cost_Trans
                CurrentLCOE=Cost/MWh
                
                Save_LCOE_Achieved.append(CurrentLCOE)
                SaveTotalMWh.append(value(MILP.OBJ))
                LowestLCOE=CurrentLCOE

                
                #Delete constraint for its modification in the next step of the for loop
                MILP.del_component(MILP.LCOE_Target)
            
            else:# Something else is wrong
                MILP.del_component(MILP.LCOE_Target)
                SaveFeasibility.append(0)
                Save_LCOETarget.append(None)
                Save_LCOE_Achieved.append(None)
                SaveYWind.append(None)
                SaveYKite.append(None)
                SaveYTrans.append(None)
                SaveCurtail.append(None)    
                SaveTotalMWh.append(None)     

                break


  0%|          | 0/225 [00:00<?, ?it/s]

Running Model With LCOE= 500.00
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3126 rows, 3678 columns and 1366798 nonzeros
Model fingerprint: 0x642592d5
Variable types: 554 continuous, 3124 integer (554 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+06]
  Objective range  [2e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2571 rows and 1336 columns
Presolve time: 1.09s
Presolved: 555 rows, 2342 columns, 705575 nonzeros
Variable types: 0 continuous, 2342 integer (552 binary)
Found heuristic solution: objective 6769010.2244

Root relaxation: cutoff, 1 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      676901

  0%|          | 1/225 [00:04<16:53,  4.52s/it]

Running Model With LCOE= 112.00
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3126 rows, 3678 columns and 1366798 nonzeros
Model fingerprint: 0xfc62d982
Variable types: 554 continuous, 3124 integer (554 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [2e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2571 rows and 1156 columns
Presolve time: 0.75s
Presolved: 555 rows, 2522 columns, 780555 nonzeros
Variable types: 0 continuous, 2522 integer (552 binary)
Found heuristic solution: objective 1444055.5145

Root relaxation: objective 6.769010e+06, 70 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6769010.22

 87%|████████▋ | 195/225 [00:08<00:01, 26.14it/s]

...

Running Model With LCOE= 76.00
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3126 rows, 3678 columns and 1366798 nonzeros
Model fingerprint: 0x4b7fd4af
Variable types: 554 continuous, 3124 integer (554 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2705 rows and 1483 columns
Presolve time: 1.91s
Presolved: 421 rows, 2195 columns, 771575 nonzeros
Variable types: 0 continuous, 2195 integer (226 binary)
Found heuristic solution: objective 243139.92872

Root relaxation: objective 2.256082e+06, 100 iterations, 0.04 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2256081.96

 95%|█████████▍| 213/225 [02:17<01:15,  6.32s/it]

Running Model With LCOE= 74.00
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3126 rows, 3678 columns and 1366798 nonzeros
Model fingerprint: 0x62138261
Variable types: 554 continuous, 3124 integer (554 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2705 rows and 1531 columns
Presolve time: 1.98s
Presolved: 421 rows, 2147 columns, 771431 nonzeros
Variable types: 0 continuous, 2147 integer (178 binary)
Found heuristic solution: objective 243139.92872

Root relaxation: objective 2.160798e+06, 29 iterations, 0.03 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2160797.68 

 95%|█████████▌| 214/225 [02:25<01:14,  6.75s/it]

Running Model With LCOE= 72.00
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3126 rows, 3678 columns and 1366798 nonzeros
Model fingerprint: 0x85fd8d50
Variable types: 554 continuous, 3124 integer (554 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2705 rows and 1540 columns
Presolve time: 2.01s
Presolved: 421 rows, 2138 columns, 771404 nonzeros
Variable types: 0 continuous, 2138 integer (169 binary)
Found heuristic solution: objective 243139.92872

Root relaxation: objective 2.073236e+06, 130 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2033479.77

In [ ]:
np.savez("./ResultsPortOpt/"+ResultsFileName +"Stage1LF"+ ".npz", 
        #Wind Data
        WindEnergy=WindEnergy,
        WindLatLong=WindLatLong,
        AnnualizedCostWind=AnnualizedCostWind,
        MaxNumWindPerSite=MaxNumWindPerSite,
        #Kite Data
        KiteEnergy=KiteEnergy,
        KiteLatLong=KiteLatLong,
        AnnualizedCostKite=AnnualizedCostKite,
        MaxNumKitesPerSite=MaxNumKitesPerSite,
        #Transmission Data
        AnnualizedCostTransmission=AnnualizedCostTransmission,
        TransLatLong=TransLatLong,
        EfficiencyTransmission=EfficiencyTransmission,
        MaxPowerTransmission=MaxPowerTransmission,
        TimeStepHours=TimeStepHours,
        #Solutions
        SaveFeasibility=SaveFeasibility,
        Save_LCOETarget=Save_LCOETarget,
        Save_LCOE_Achieved=Save_LCOE_Achieved,
        SaveYWind=SaveYWind,
        SaveYKite=SaveYKite,
        SaveYTrans=SaveYTrans,
        SaveCurtail=SaveCurtail,
        SaveTotalMWh=SaveTotalMWh)


/home/victor-duraes/anaconda3/envs/Gurobi_py37/lib/python3.7/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
